In [1]:
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import scipy.sparse as sp
import time
import random
from concurrent.futures import ProcessPoolExecutor as prpExecutor
import sys

In [47]:
# ============================
# Data File Path
# ============================
TRAIN_DATA_FILE_PATH = "/hdd1/Spoiler_Detection/ACL/INGGEOL/train_10000.json"
VALIDATION_DATA_FILE_PATH = "/hdd1/Spoiler_Detection/ACL/INGGEOL/validation_10000.json"
PRE_TRAINED_WORD_EMBEDDING_FILE_PATH = "word_embedding_10000.npy"

# ============================
# Model Hyper Parameter
# ============================
EMBEDDING_DIM = 50
HIDDEN_STATES = [50, 50]

# ============================
# Training Hyper Parameter
# ============================
EPOCHS = 300
LEARNING_RATE = 0.00005
BATCH_SIZE = 256
WEIGHT_DECAY = 1e-5
DROPOUT_RATE = 0.5
RANDOM_SEED = 26npy

SyntaxError: invalid syntax (<ipython-input-47-65e59ef280cb>, line 22)

In [36]:
# ============================
# Data Pre-Processing
# ============================
def load_data(train_file_path, validation_file_path):
    with open(train_file_path) as f:
        train = json.load(f)["data"]
    with open(validation_file_path) as f:
        validation = json.load(f)["data"]
        
    return train, validation
    
def make_dictionary(train, validation):
    data = []
    data += train
    data += validation
    
    maximum_length = max([len(line.split("\t")[0].split()) for line in data])

    word2id = {"<PAD>":0}
    id2word = ["<PAD>"]
    edge2id = {}
    id2edge = []
    
    for line in data:
        tokens = line.split("\t")
        for word in tokens[0].split():
            if word not in word2id:
                word2id[word] = len(word2id)
                id2word.append(word)
        for edges in tokens[2:]:
            _tokens = edges.split(":")
            if len(_tokens) != 3:
                start, end = _tokens[0], _tokens[1]
                edge = ":".join(_tokens[2:])
            else:
                start, end, edge = _tokens
            if edge not in edge2id:
                edge2id[edge] = len(edge2id)
                id2edge.append(edge)

    return word2id, id2word, edge2id, id2edge, maximum_length

def make_input_data_as_index(_train, _validation, word2id, edge2id):
    train, validation = [], []
    for line in _train:
        tokens = line.split("\t")
        tokens[0] = [word2id[word] for word in tokens[0].split()]
        _edges = []
        for edges in tokens[2:]:
            _tokens = edges.split(":")
            if len(_tokens) != 3:
                start, end = _tokens[0], _tokens[1]
                edge = ":".join(_tokens[2:])
            else:
                start, end, edge = _tokens
            _edges.append([start, end, edge2id[edge]])
        train.append([tokens[0], tokens[1], _edges])

    for line in _validation:
        tokens = line.split("\t")
        tokens[0] = [word2id[word] for word in tokens[0].split()]
        _edges = []
        for edges in tokens[2:]:
            _tokens = edges.split(":")
            if len(_tokens) != 3:
                start, end = _tokens[0], _tokens[1]
                edge = ":".join(_tokens[2:])
            else:
                start, end, edge = _tokens
            _edges.append([start, end, edge2id[edge]])
        validation.append([tokens[0], tokens[1], _edges])

    return train, validation

def make_input_adjacency_matrix(line):
    words, label, edges = line[0], float(line[1]), line[2]
    adjacency_matrix = make_adjacency_matrix(np.asarray(edges), len(words))
    
    return [words, adjacency_matrix, label]

def normalize_matrix(matrix):
    rowsum = np.asarray(matrix.sum(1))
    row_inv = np.power(np.sqrt(rowsum), -1).flatten()
    row_inv[np.isinf(row_inv)] = 0.
    row_matrix_inv = sp.diags(row_inv)
    matrix = row_matrix_inv.dot(matrix)

    return matrix

def sparse_matrix_to_torch_sparse_tensor(sparse_matrix, maximum_length):
    sparse_matrix = sparse_matrix.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_matrix.row, sparse_matrix.col)).astype(np.int64))
    values = torch.from_numpy(sparse_matrix.data)
    shape = torch.Size((maximum_length, maximum_length))

    return torch.sparse.FloatTensor(indices, values, shape)

def make_adjacency_matrix(edges, num_words):
    adjacency_matrix = sp.coo_matrix(
        (np.ones(len(edges)), (edges[:, 0].astype(np.int32), edges[:, 1].astype(np.int32))),
        shape=(num_words, num_words),
        dtype=np.float32
    )
    
    identity_matrix = sp.coo_matrix(
        (np.ones(len(edges)), (np.arange(len(edges)), np.arange(len(edges)))),
        shape=(num_words, num_words),
        dtype=np.float32
    )
    
    adjacency_matrix = adjacency_matrix + identity_matrix + adjacency_matrix.transpose()
    normalized_adjacency_matrix = normalize_matrix(adjacency_matrix)
    
    return normalized_adjacency_matrix

def load_pre_trained_word_embedding(word_embedding_file_path):
    return torch.from_numpy(np.load(word_embedding_file_path).astype(np.float32))

def make_batch(data, batch_size, is_train=True):
    indices = np.arange(len(data))
    if is_train:
        random.shuffle(indices)
    
    if len(data) % batch_size == 0:
        batch_num = int(len(data)/batch_size)
    else:
        batch_num = int(len(data)/batch_size) + 1
        
    for i in range(batch_num):
        left = i*batch_size
        right = min((i+1)*batch_size, len(data))
        
        sentences = []
        adjacency_matrics = []
        labels = []
        
        for j in indices[left:right]:
            sentences.append(data[j][0])
            adjacency_matrics.append(data[j][1])
            labels.append(data[j][2])
        
        yield sentences, adjacency_matrics, labels

In [9]:
# ============================
# Data Pre Processing
# ============================
print("Load Data...")
train, validation = load_data(TRAIN_DATA_FILE_PATH, VALIDATION_DATA_FILE_PATH)

print("Make Dictionary...")
word2id, id2word, edge2id, id2edge, maximum_length = make_dictionary(train, validation)

print("Make Input as Index...")
train, validation = make_input_data_as_index(train, validation, word2id, edge2id)

print("Make Adjacency Matrix...")
start = time.time()
pool = prpExecutor(max_workers=16)
train = list(pool.map(make_input_adjacency_matrix, train))
validation = list(pool.map(make_input_adjacency_matrix, validation))
del pool
print(int(time.time() - start))

print("Make Sparse Tensor...")
start = time.time()
for line in train+validation:
    line[0] += [0] * (maximum_length - len(line[0]))
    line[1] = sparse_matrix_to_torch_sparse_tensor(line[1], maximum_length)
print(int(time.time() - start))

print("Load Pre-trained Word Embedding...")
word_embedding = load_pre_trained_word_embedding(PRE_TRAINED_WORD_EMBEDDING_FILE_PATH)

Load Data...
Make Dictionary...
Make Input as Index...
Make Adjacency Matrix...


/home/buru/python3_venv/lib/python3.5/site-packages/ipykernel_launcher.py:83: RuntimeWarning: divide by zero encountered in power
/home/buru/python3_venv/lib/python3.5/site-packages/ipykernel_launcher.py:83: RuntimeWarning: divide by zero encountered in power
/home/buru/python3_venv/lib/python3.5/site-packages/ipykernel_launcher.py:83: RuntimeWarning: divide by zero encountered in power
/home/buru/python3_venv/lib/python3.5/site-packages/ipykernel_launcher.py:83: RuntimeWarning: divide by zero encountered in power
/home/buru/python3_venv/lib/python3.5/site-packages/ipykernel_launcher.py:83: RuntimeWarning: divide by zero encountered in power
/home/buru/python3_venv/lib/python3.5/site-packages/ipykernel_launcher.py:83: RuntimeWarning: divide by zero encountered in power
/home/buru/python3_venv/lib/python3.5/site-packages/ipykernel_launcher.py:83: RuntimeWarning: divide by zero encountered in power
/home/buru/python3_venv/lib/python3.5/site-packages/ipykernel_launcher.py:83: RuntimeWarni

58
Make Sparse Tensor...
32


In [38]:
print("Load Pre-trained Word Embedding...")
word_embedding = load_pre_trained_word_embedding(PRE_TRAINED_WORD_EMBEDDING_FILE_PATH)

Load Pre-trained Word Embedding...


In [39]:
# ============================
# Model
# ============================
class GCNLayer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(GCNLayer, self).__init__()

        self.input_dim = input_dim
        self.output_dim = output_dim

        self.weight = nn.Parameter(
            torch.randn(self.input_dim, self.output_dim))
        nn.init.xavier_normal_(self.weight)

        self.bias = nn.Parameter(torch.zeros(self.output_dim))

    def forward(self, x, adj_matrics):
        x = torch.matmul(adj_matrics, x)
        output = torch.matmul(x, self.weight)
        output = output + self.bias

        return output

class Model(nn.Module):
    def __init__(self, num_words, embedding_dim, hidden_dim, maximum_length, pre_trained, dropout_rate):
        super(Model, self).__init__()

        self.num_words = num_words
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.maximum_length = maximum_length
        self.dropout_rate = dropout_rate

        # =============================================
        # Data Preparation
        # =============================================		
        self.word_embedding \
        = nn.Embedding(self.num_words, self.embedding_dim, padding_idx = 0)
        self.word_embedding.from_pretrained(pre_trained)
        
        self.gcn_layer_1 \
        = GCNLayer(self.embedding_dim, self.hidden_dim[0])
        self.gcn_layer_2 \
        = GCNLayer(self.hidden_dim[0], self.hidden_dim[1])
        
        self.max_pooling = nn.MaxPool1d(self.maximum_length)
        self.output_layer = nn.Linear(self.hidden_dim[1], 1)

    def forward(self, sentences, adjacency_matrics, batch_size):
        embedded_words = self.word_embedding(sentences)
        gcn_1 = self.gcn_layer_1(embedded_words, adjacency_matrics)
        gcn_1 = F.relu(gcn_1)
        gcn_1 = F.dropout(gcn_1, self.dropout_rate)
        gcn_2 = self.gcn_layer_2(gcn_1, adjacency_matrics)
        gcn_2 = F.relu(gcn_2)
        sentence_representations = self.max_pooling(gcn_2.transpose(1, 2)).squeeze()
        sentence_representations = F.dropout(sentence_representations, self.dropout_rate)
        output = self.output_layer(sentence_representations)
        
        return output

In [45]:
# =============================================
# Model Initialize
# =============================================
print("Model Initializing..")
pos_weight = 20*torch.ones([1]).cuda()
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

model = Model(len(word2id), EMBEDDING_DIM, HIDDEN_STATES, maximum_length, word_embedding, DROPOUT_RATE).cuda()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

Model Initializing..


In [ ]:
# =============================================
# Model Training
# =============================================
print("Model Training..\n")
for i in range(EPOCHS):
    model.train()
    start = time.time()
    
    if len(train) % BATCH_SIZE == 0:
        batch_num = int(len(train)/BATCH_SIZE)
    else:
        batch_num = int(len(train)/BATCH_SIZE) + 1
    
    loss = .0
    batches = make_batch(train, BATCH_SIZE)
    step = 0

    for batch in batches:
        sentences, adjacency_matrics, labels = batch
        input_sentences = torch.tensor(sentences, dtype = torch.long).cuda()
        input_adjacency_matrics = torch.stack([matrix.to_dense() for matrix in adjacency_matrics], dim=0).cuda()
        input_labels = torch.tensor(labels, dtype=torch.float32).unsqueeze(dim=1).cuda()
        optimizer.zero_grad()
        logits = model(input_sentences, input_adjacency_matrics, len(sentences))
        _loss = criterion(logits, input_labels).sum()
        _loss.backward()
        optimizer.step()
        loss += _loss.item()
        step+=1
        
        sys.stdout.flush()
        sys.stdout.write("\r" + "Epoch: [{}/{}] Batch: [{}/{}]".format(i+1, EPOCHS, step, batch_num))
    
    if (i+1) % 1 == 0:
        model.eval()
        
        batches = make_batch(validation, BATCH_SIZE)
        if len(validation) % BATCH_SIZE == 0:
            batch_num = int(len(validation)/BATCH_SIZE)
        else:
            batch_num = int(len(validation)/BATCH_SIZE) + 1
        
        step = 0
        count = 0
        correct = 0
        positive_answer = 0
        positive_actual = 0
        for batch in batches:
            sentences, adjacency_matrics, labels = batch
            input_sentences = torch.tensor(sentences, dtype = torch.long).cuda()
            input_adjacency_matrics = torch.stack([matrix.to_dense() for matrix in adjacency_matrics], dim=0).cuda()
            input_labels = torch.tensor(labels, dtype=torch.float32).unsqueeze(dim=1).cuda()
            logits = model(input_sentences, input_adjacency_matrics, len(sentences))

            predicted = (logits > 0.5).float()
            positive_answer += predicted.sum().item()
            positive_actual += (input_labels == 1.0).float().sum().item()
            correct+=(predicted*input_labels).sum().item()
            count+= (predicted==input_labels).sum().item()
            step+=1

        accuracy = 100*float(count)/len(validation)
        if positive_answer == 0:
            precision = 0.0
        else:
            precision = float(correct)/positive_answer
        recall = float(correct)/positive_actual
        if (precision+recall) == 0.0:
            f1 = 0.0
        else:
            f1 = 2*precision*recall/(precision+recall)
        print(" Loss: {} [A/P/R/F]: [{:.4f}/{:.4f}/{:.4f}/{:.4f}]".format(loss, accuracy, precision, recall, f1))
    
    else:
        print(" Loss: {}".format(loss))

Model Training..

Epoch: [1/100] Batch: [463/463] Loss: 196.2494596838951 [A/P/R/F]: [90.9312/0.1008/0.3303/0.1545]
Epoch: [2/100] Batch: [463/463] Loss: 194.7464906424284 [A/P/R/F]: [91.6458/0.1120/0.3364/0.1681]
Epoch: [3/100] Batch: [463/463] Loss: 193.59879004955292 [A/P/R/F]: [91.6458/0.1048/0.3091/0.1566]
Epoch: [4/100] Batch: [463/463] Loss: 192.20072720944881 [A/P/R/F]: [91.7218/0.1043/0.3030/0.1552]
Epoch: [5/100] Batch: [463/463] Loss: 194.18502932786942 [A/P/R/F]: [91.2125/0.0967/0.3000/0.1462]
Epoch: [6/100] Batch: [463/463] Loss: 190.5301441848278 [A/P/R/F]: [91.8814/0.1148/0.3333/0.1708]
Epoch: [7/100] Batch: [463/463] Loss: 189.88383451104164 [A/P/R/F]: [91.4177/0.1095/0.3394/0.1656]
Epoch: [8/100] Batch: [463/463] Loss: 191.2514629662037 [A/P/R/F]: [91.4481/0.1084/0.3333/0.1636]
Epoch: [9/100] Batch: [463/463] Loss: 188.09292414784431 [A/P/R/F]: [91.3265/0.1021/0.3152/0.1542]
Epoch: [10/100] Batch: [463/463] Loss: 189.28051498532295 [A/P/R/F]: [91.6306/0.1070/0.3182/0.1

In [12]:
count = 0
for line in train:
    if line[2] == 1.0:
        count+=1
print(100*float(count)/len(train))

count = 0
for line in validation:
    if line[2] == 1.0:
        count+=1
print(100*float(count)/len(validation))

2.6683673038424827
2.508551881413911


13155